## 머신러닝 워크플로우 최적화

- 파이프라인을 사용하여 여러 추정기를 하나로 묶을 수 있다.
- 편의성과 캡슐화 

# ML Workflow Optimization
<font color=#CC3D3D>
## Pipeline: chaining estimators   
</font>
- Pipeline can be used to chain multiple estimators into one.
- Pipeline serves two purposes:
  - Convenience and encapsulation
  - Joint parameter selection
- All estimators in a pipeline, except the last one, must be transformers. 
  - The last estimator may be any type (transformer, classifier, etc.)
- Training and prediction procedure of the pipeline
<br>
<img align="left" src="http://drive.google.com/uc?export=view&id=1pIde-P6d7EnjL3xYo8eE3cWAUEvzV7tS" >

In [ ]:
import pandas as pd
import copy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=0)   

### Building Pipelines

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

** 파이프라인은 (키, 값) 목록을 사용하여 빌드됩니다.** 여기서 key 는 이 단계에서 지정할 이름을 포함하는 문자열이고 value는 추정기 개체입니다.

In [ ]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])

- 전체 일련의 추정기에 적합하도록 데이터에 대해  fit 및 predict 를 한 번만 호출하면 됩니다.

In [ ]:
pipe.fit(X_train, y_train).score(X_test, y_test)

### Using Pipelines in Grid-searches

In [ ]:
from sklearn.model_selection import GridSearchCV # 최적의 파라미터 검색

파이프라인의 추정기 매개 변수는 ** 추정기__매개변수** 구문을 사용하여 정의해야 합니다.

In [ ]:
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(
    grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

### *make_pipeline*을 통한 편리한 파이프라인 생성

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
# standard syntax
pipe_long = Pipeline([("scaler", MinMaxScaler()), 
                      ("svm", SVC(C=100))])
#pipe_long = Pipeline([('pca', PCA(n_components=3))), 
 #                     (('univ_select', SelectKBest(k=10))])

In [ ]:
# abbreviated syntax(약칭구문 - 이름을 넣지 않고 파이프 라인 만들기)
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100)) 

**Make_pipeline**은(는) 추정기의 이름을 지정할 필요가 없으며 허용하지 않습니다. <br>
대신, 이름은 ** 소문자*로 자동 설정됩니다.

In [ ]:
print("Pipeline steps:\n{}".format(pipe_short.steps))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipe = make_pipeline(StandardScaler(), PCA(n_components=2), 
                     StandardScaler())
print("Pipeline steps:\n{}".format(pipe.steps))

*FeatureUnion*과 기능 결합

<img align='left' src='https://image.slidesharecdn.com/featureengineeringpipelines1-161106200348/95/feature-engineering-pipelines-11-638.jpg?cb=1478462927' width=600 height=400>

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA # 차원축소 
from sklearn.feature_selection import SelectKBest # 성능이 좋은 변수만 사용하는 전처리기

In [ ]:
# create feature union
features = []
features.append(('pca', PCA(n_components=3)))
features.append(('univ_select', SelectKBest(k=10)))
feature_union = FeatureUnion(features)

# create pipeline
estimators = []
estimators.append(('features', feature_union))
estimators.append(('scaler', MinMaxScaler()))
estimators.append(("svm", SVC()))
pipe = Pipeline(estimators)

In [ ]:
pipe.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
# Do grid search
param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[9, 10, 11],
                  svm__C=[0.1, 1, 10],
                  svm__gamma=[0.1, 1, 10])
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=5)
print(grid_search.fit(X_train, y_train).score(X_test, y_test))
print(grid_search.best_estimator_)

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv('titanic_train.csv')
train.head()

#### 타이타닉 호 침몰 당시의 승객 명단 데이터

- Survived: 생존 여부 => 0 = No, 1 = Yes
- pclass: 티켓 등급 => 1 = 1st, 2 = 2nd, 3 = 3rd
- Sex: 성별
- Age: 나이
- Sibsp: 함께 탑승한 형제자매, 배우자의 수
- Parch: 함께 탑승한 부모, 자식의 수
- Ticket: 티켓 번호
- Fare: 운임
- Cabin: 객실 번호
- Embarked: 탑승 항구 => C = Cherbourg, Q = Queenstown, S = Southampton

In [ ]:
train.info()

In [ ]:
train.iloc[:,2:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train.iloc[:,2:], train.Survived, random_state=0)

#### 전처리 방향
feature를 `수치형과 범주형으로 나누어`<sup>1)</sup> 다르게 전처리를 수행한다.
- 수치형의 경우: 결측값을 중앙값으로 대체 -> Standardization
- 범주형의 경우: `결측값이 없는 모든 범주형 feature에 대해 One-Hot-Encoding 수행`<sup>2)</sup>


*<sup>1),2)</sup> scikit-learn에 없는 전처리 기능이기 때문에 Custom Transformer를 만들어야 함.*

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer

(참고) https://databuzz-team.github.io/2018/11/11/make_pipeline/

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# BaseEstimator를 상속하면 get_params 및 set_params를 무료로 얻을 수 있습니다
# TransformerMixin를 상속하면 fit 및 transform 메서드를 작성하고 fit_transform을 무료로 얻을 수 있습니다
# (참고) https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65

# 1)번 Custom Transformer
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.feature_names].values

In [ ]:
# 2)번 Custom Transformer
class CustomLabelBinarizer(BaseEstimator, TransformerMixin):
    def fit(self, X_cat, y=None):
        return self
    def transform(self, X_cat):
        X_cat_df = pd.DataFrame(X_cat, columns=range(X_cat.shape[1]))
        X_onehot_df = pd.get_dummies(X_cat_df, columns=X_cat_df.columns)
        return X_onehot_df.values

In [ ]:
# feature를 수치형과 범주형으로 구분
con = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] # 수치형 변수들
cat = ['Sex', 'Ticket']  # 범주형 변수들

In [ ]:
# 인덱스 위치
con_idx = [train.columns.get_loc(c) for c in train.columns if c in con]
cat_idx = [train.columns.get_loc(c) for c in train.columns if c in cat]

In [ ]:
print(con_idx, cat_idx)

In [ ]:
# 수치형 feature에 대한 전처리
con_pipeline = Pipeline([
    ('selector', DataFrameSelector(con)),
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
])

# 범주형 feature에 대한 전처리
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat)),
    ('encoder', CustomLabelBinarizer()),
])

# 전처리된 수치형과 범주형 feature를 결합
full_pipeline = FeatureUnion([
    ('con_pipeline', con_pipeline),
    ('cat_pipeline', cat_pipeline),
])

In [ ]:
print("con_pipeline steps:\n{}".format(con_pipeline.steps))

In [ ]:
print("cat_pipeline steps:\n{}".format(cat_pipeline.steps))

In [ ]:
full_pipeline

In [ ]:
X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)